# Import Packages

In [43]:
import os
import re
import numpy as np
import pandas as pd
from collections import Counter
from transformers import (AutoTokenizer,)
from datasets import Dataset, DatasetDict, Features, ClassLabel, Value, concatenate_datasets
from huggingface_hub import login
from collections import Counter

import utility.utility as util

# Below import and instructions simply for display
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# resets import once changes have been applied
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Directory and File Paths

In [44]:
path_cwd = os.getcwd()

path_datasets = os.path.join(path_cwd, "datasets")

# Configuration

In [45]:
_from_hub = False

_dataset_names_hub = ["Databasesprojec/FinStmts_ConsUncons_English_EU_Predict_part_1",
                      "Databasesprojec/FinStmts_ConsUncons_English_EU_Predict_part_2"]

_dataset_names_local = ["test_english_new"]

_col_text = "text"
_col_id = "filename"
_col_label = "label"

_base_model = "bert-base-multilingual-cased"

_flag_full_seg = True

_semi_num_segments = {"front":4, "back":4}

_slide_step_size_perc_of_max = .2

paths_local_datasets = [os.path.join(path_datasets, ds) for ds in _dataset_names_local]
datasets = _dataset_names_hub if _from_hub else paths_local_datasets

# Load Tokenizer

In [46]:
tokenizer = AutoTokenizer.from_pretrained(_base_model)

# Extract Info

In [47]:
no_spec_tokens_per_example = util.det_num_spec_tokens(tokenizer)

tokens_per_segment = tokenizer.model_max_length - no_spec_tokens_per_example

slide_step_size = int(tokens_per_segment * _slide_step_size_perc_of_max)

special_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.all_special_tokens)

# Segment and Save DatasetDicts

In [48]:
for ds in datasets:
    tmp_data = util.load_data(_from_hub, ds, ds)
    features = None
    for key in tmp_data:
        features = tmp_data[key].features
        features["id"] = Value("string")
        
    tmp_data = util.apply_sliding_window_to_datasetdict(tmp_data,
                                                        _col_id,
                                                        _col_text,
                                                        tokenizer, 
                                                        tokens_per_segment, 
                                                        special_token_ids,
                                                        slide_step_size,
                                                        _col_label,
                                                        _semi_num_segments,
                                                        _flag_full_seg)

    tmp_data = util.create_dataset_dict(tmp_data, features)

    tmp_data.save_to_disk(ds + "_segmented_" + _base_model + ("_full" if _flag_full_seg else "_semi"))        

DatasetDict({
    train: Dataset({
        features: ['filename', 'text', 'label'],
        num_rows: 9987
    })
    val: Dataset({
        features: ['filename', 'text', 'label'],
        num_rows: 2134
    })
    test: Dataset({
        features: ['filename', 'text', 'label'],
        num_rows: 2138
    })
})
train


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9987/9987 [46:12<00:00,  3.60it/s]


val


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2134/2134 [1:16:06<00:00,  2.14s/it]


test


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2138/2138 [07:28<00:00,  4.77it/s]


Saving the dataset (0/1 shards):   0%|          | 0/79707 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17064 [00:00<?, ? examples/s]